In [1]:
import gensim
import gensim.corpora as corpora
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models
import seaborn as sns
import tqdm
from gensim.models import CoherenceModel
from wordcloud import WordCloud

from data_storage import retrieve_twitter_data

/home/devilgoncalo/miniconda3/envs/dataalliance/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
df = retrieve_twitter_data()

#### Create corpus, dictionary

In [3]:
# Create Corpus
texts = df['lemmatized_text'].values

# Create Dictionary
id2word = corpora.Dictionary(texts)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# LDA
Hyperparameters:
- Alpha - higher means documents are assumed to be made up of more topics - try 0.1
- Beta - controls distribution of words per topic, higher means topics have more words

*default = 1.0 for both*

In [8]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=6, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=2,
                                       per_word_topics=True)

#### Coherence scores
The overall coherence score of a topic is the average of the distances between words.
For u_mass the closer to 0 the better. It is a logarithmic function.

In [9]:
# Compute Coherence Score - 'c_v' is the best coherence score, but slow
#                         - 'u_mass' requires corpus
coherence_model_lda = CoherenceModel(model=lda_model,
                                     corpus=corpus,
                                     texts=texts,
                                     dictionary=id2word,
                                     coherence='u_mass') 

coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)


Coherence Score:  -3.7709936395099586


#### Visualization

In [10]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/home/devilgoncalo/miniconda3/envs/dataalliance/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/devilgoncalo/miniconda3/envs/dataalliance/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/devilgoncalo/miniconda3/envs/dataalliance/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/devilgoncalo/miniconda3/envs/dataalliance/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.205596  0.006546       1        1  20.933106
1     -0.047577  0.068934       2        1  20.210591
2      0.080043  0.158061       3        1  17.375923
5     -0.204396 -0.088288       4        1  15.341729
3      0.185486 -0.278075       5        1  13.364918
0      0.192040  0.132822       6        1  12.773733, topic_info=            Term          Freq         Total Category  logprob  loglift
168      vaccine  14573.000000  14573.000000  Default  30.0000  30.0000
85          case   8755.000000   8755.000000  Default  29.0000  29.0000
130         test   9409.000000   9409.000000  Default  28.0000  28.0000
180          new  10398.000000  10398.000000  Default  27.0000  27.0000
74           get  16870.000000  16870.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
492     pandemic   1120.506902   6452.213308   Topic6  -5.1292   0.3071
30          year   1161.936855   9287.008981   Topic6  -5.0929  -0.0207
380   government    899.318190   2874.970596   Topic6  -5.3491   0.8956
684        cause    822.671469   2773.975092   Topic6  -5.4382   0.8423
1129        call    764.818332   1639.965988   Topic6  -5.5111   1.2950

[418 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
4123      6  0.996792  accept
747       3  0.997449  active
1576      5  0.972836     add
1576      6  0.025981     add
2330      2  0.998892   admit
...     ...       ...     ...
30        4  0.063314    year
30        5  0.106062    year
30        6  0.125121    year
10        2  0.998330  you_re
10        3  0.001420  you_re

[794 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 6, 4, 1])

#### Interpretation of results per topic
1. Covid health (symptom, feel, bad, long, sick)
2. Covid politics (government, public, official, lockdown, people)
3. Covid measures (mask, wear, infection, high, rate, variant)
4. Unninterpretable
5. Covid vaccine (vaccine, child, booster, risk, dose, study)
6. Covid results (test, case, positive, death, new, day, report, daily, rise)
7. Covid conspiracy (virus, tell, try, lie, wonder, policy, fake)
8. Unninterpretable
9. Covid vaccine (vaccine, update, kid, young, vaccination, booost, pfizer)
10. Unninterpretable

## Save the topics and interpretation in the dataframe

In [42]:
df_copy = df.copy()

In [40]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    return(sent_topics_df)

df_dominant_topic = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df["lemmatized_text"])

# Format
df_dominant_topic = df_dominant_topic.reset_index()
df_dominant_topic.drop(['index'], axis=1, inplace=True)

/tmp/ipykernel_15308/2440210496.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/tmp/ipykernel_15308/2440210496.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [61]:
# merge dataframes
final_df = pd.concat([df_copy.reset_index(drop=True), df_dominant_topic.reset_index(drop=True)], axis=1)

## Save final dataframe

In [65]:
final_df.to_parquet(os.getcwd()+'/data_storage/data/twitter_data.parquet.gzip', compression='gzip')

# Hyperparameter tuning

#### Elbow method to decide K number of clusters

In [4]:
from gensim.models.ldamodel import LdaModel

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute u_mass coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='u_mass')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=20, step=4)

# Show graph
import matplotlib.pyplot as plt
limit=20; start=2; step=4;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

#### Tune parameters alpha and eta

In [5]:
# Set number of topics based on elbow method result
k = 6

In [6]:
def optimize_lda(k):
    results = []

    # Alpha parameter
    alpha = list(np.arange(0.01, 1, 0.3))
    alpha.append('symmetric')
    alpha.append('asymmetric')

    # Beta parameter
    beta = list(np.arange(0.01, 1, 0.3))
    beta.append('symmetric')
    
    for a in alpha:
        for b in beta:
            lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                    id2word=id2word,
                                    num_topics=k, 
                                    random_state=100,
                                    chunksize=100,
                                    passes=2,
                                    per_word_topics=True,
                                    alpha=a,
                                    eta=b
                                    )

            coherence_model_lda = CoherenceModel(model=lda_model,
                                                corpus=corpus,
                                                texts=texts,
                                                dictionary=id2word,
                                                coherence='u_mass') 

            # get coherence value
            coherence_lda = coherence_model_lda.get_coherence()

            # save results
            results_dict = {'alpha': a,
                            'beta': b,
                            'K': k,
                            'Coherence score': coherence_lda}
            
            results.append(results_dict)
    
    return results

In [7]:
optimization_results = optimize_lda(6)

/home/devilgoncalo/miniconda3/envs/dataalliance/lib/python3.9/site-packages/astroid/node_classes.py:94: DeprecationWarning: The 'astroid.node_classes' module is deprecated and will be replaced by 'astroid.nodes' in astroid 3.0.0
  warnings.warn(


NotImplementedError: auto-tuning alpha not implemented in LdaMulticore; use plain LdaModel.

#### Save model

In [ ]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("model")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

#### More hyperparameter tunning (don't have the machine for it)

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=2,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model,
                                        corpus=corpus,
                                        texts=texts,
                                        dictionary=id2word,
                                        coherence='u_mass')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                k=k, a=a, b=b)
                # Save the model results
                model_results['Validation_Set'].append(corpus_title[i])
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
    pd.DataFrame(model_results).to_csv(os.getcwd()+'/models/lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
# Read results
df_results = pd.read_csv(os.getcwd()+'/models/lda_tuning_results.csv')

df_results